# Business Problem Statement:
A group of customers were given an offer in person that they can get a loan at discounted rate and
processing fee will be waived off. A pilot campaign was conducted to get response from customers
whether they are interested in taking out a loan or not. Response was recorded and data was collected.

### Task List

- [x] Build a model to predict whether customers will be interested in taking out a loan or not.
- [x] Identifying features which are most important
- [x] In case of black box models e.g. Random forest use SHAP, LIME to figure out features affecting the target variable
- [x] Try Unsupervised clustering models
- [x] Remove Unnecessary Models from the File
- [x] Generate synthetic data for model.
- [x] Approaching a customer has costs involved with it, hence find the profitable segments so that more customized marketing can be done.
- [x] Need to write inferences what is going on
- [x] Matrix evaluation for all 20% above models 
- [x] KS Scaling
- [x] Variance Inflation Factor (VIF)
- [x] Bucketing Age and SCR
- [x] Business Output

Variables involved: `Customer_id`, `Age`, `Gender`, `Balance`, `Occupation`, `No of Credit transaction`, `SCR`, `Holding period`

## Understanding Complex Variables


- `Holding Period` (How long the customer is able to hold the money in his account.. So, if they have some existing expenses like a loan EMI or any other monthly expense which gets deducted, usually the first week of every month, hence it makes the balance in the account lower during initial days of the month itself.Higher the holding period, more stable their money is in the account.)

- `SCR` SCR is a score given to a customer for a particular product ( in this case loan ) based on certain parameters, to know whether how likely that customer is to buy that product.. so, higher the score, higher the probability, the customer will buy it. SCR propensity of a customer to respond to a digital marketing


## Business Conclusion

#####  HOLDING PERIOD Vs AVG BALANCE

1. The avg holding period and avg balance tend to show high collinearity as they are directly proportional and it can be verified from "Misc_Dash".

2. Self employed class in both genders were able to hold money in their account for a long time resulting in high balance amount.

##### Which Category of Customers Tend to opt for Loans?

Customers with the below traits tend to be inclined towards opting for a loan;

1. Females who own small business/self-owned (SENP) business show higher SCR.

2. Males who are self employed (SELF-EMP) show higher SCR .

3. High SCR's but least holding period i.e. who tend to have EMI's or other deductions from their account.

4. Higher Avg credit transactions shows that the customer is more likely to opt for a loan. As Credit transactions show direct proportionality to SCR values.

##### Final Conclusions

1. From the given data it can be observed that in Females Self Employed tend to be the early bread earners whereas in case of Males Small Business owners tend to be the early bread earners.

2. For both the genders higher avg balance amount is observed in the accounts of Small business owners, second highest avg balance is observed for Professionals (PROF) category customers.


##### Which Category of Customers TEND NOT TO OPT for Loans?

1. In Males, Salaried & Professional class employees tend to not opt for loans as they must have constant inflow of cash.

2. In Females, Professional class employees tend to not opt for loans.

3. In both the genders, customers showing higher holding period tend to not opt for loans as there are less deductions over long time periods.

### Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import statsmodels.api as sm

from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score, roc_curve, auc, plot_roc_curve
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import plot_partial_dependence
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import SMOTE
from statsmodels.stats.outliers_influence import variance_inflation_factor

### Reading Data from CSV file

In [ ]:
data = pd.read_csv('Model_data.csv')

In [ ]:
data.Balance = data.Balance.astype('int32') #Truncating decimals

In [ ]:
data.head()

In [ ]:
data.Balance.describe()

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.describe()

`Gender` and `Occupation` are categorical varibles stored as object type

### ------------------------------------------------------Start of Exploratory Data Analysis-------------------------------------------------------

In [ ]:
sns.heatmap(data.corr(), annot=True, square=True) # No strong correlations seen overall
plt.show()

###### Inferences: 

- `TARGET` does show maximum correlation w.r.t `No_OF_CR_TXNS` among other variables. Remaining correlations have to be visualized in order to understand if there's any sort of trend in data.

In [ ]:
# sns.pairplot(data, hue='Target')
# plt.show()

##### Inferences from Pair-Plot: 

- People who tend to opt for loan have less `BALANCE` w.r.t people not buying loan.
- People having higher `BALANCE` tend to be in between 35-50 years of `AGE`.
- People having less `BALANCE` tend to show more interest towards loans i.e. higher `SCR` values.
- People having less `BALANCE` tend to have higher `No of Credit transactions` opt for loans.
- People having less `BALANCE` have higher `HOLDING PERIOD` who opt for loans.

In [ ]:
sns.countplot(x = data.Gender)
plt.show()

###### Observation: 

- Since `GENDER` = `"OTHERS"/"O"` has very less population size i.e. less than 1% of entire data hence can be omitted as an attribute from `GENDER` category.

In [ ]:
data.Gender.unique()

In [ ]:
data.Occupation.unique()

In [ ]:
data.Gender.value_counts()

In [ ]:
data.drop(data.Gender[data.Gender== 'O'].index, axis = 0, inplace= True) # Removed 196 rows with `Gender` = 'O'

In [ ]:
data.shape

In [ ]:
data.Balance.describe()

In [ ]:
# sns.histplot(data.Age)

In [ ]:
sns.countplot(x = data.Occupation)

###### Inferences: 

- The data consists mostly of `Salaried` category employees, whereas `Business Owners` and `Professionals` occupy the same strength in the data.

In [ ]:
sns.countplot(x=data.Target, hue=data.Occupation) ## Self employed are much more likely to take loans

##### Inferences: 

- Most people who opted for loans tend to be either `Self-Employed` or `Small businesses` whereas `Salaried` people showed quite less interest in taking up loans since they had fixed income.

In [ ]:
g = sns.FacetGrid(data, col='Occupation', hue="Gender")
plt.grid(True)
g.map(sns.countplot, "Gender", alpha=1)
g.add_legend()
plt.grid((False))

###### Inferences: 

- Most of the `Males` tend to be either in `Salaried` employee class (nearly 85%) or `working professionals` (nearly 70%) whereas most `Women` (nearly 90%) tend to start-up their own `business's` and manage them.

--------------------------

In [ ]:
# sns.histplot(data.No_OF_CR_TXNS)

In [ ]:
data.No_OF_CR_TXNS.describe()

In [ ]:
sns.violinplot(x=data.No_OF_CR_TXNS)
plt.grid(True)

###### Inferences: 

- The average `No. of Credit Transactions` tend to be in between 10 and 20. Whereas very few transactions tend to be in the number of 30-50.  

In [ ]:
# len(data[data.No_OF_CR_TXNS==0].index)

In [ ]:
# data.drop(index=data[data.No_OF_CR_TXNS==0].index, axis=0)

------------------

In [ ]:
# sns.displot(data.SCR, kind = 'kde')
sns.distplot(data.SCR)
plt.show()

###### Inferences: 

- A uniform distribution can be observed for `SCR` and no skewness is found in the given data. 

In [ ]:
data.SCR.describe()

In [ ]:
sns.distplot(data.Holding_Period)

In [ ]:
data3 = data

### ------------------------------------------------------End of Exploratory Data Analysis-------------------------------------------------------
-----------
----------

### Pretty Report Printing Function

In [ ]:
# A class for pretty printing
class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'
    
# function for validation on test data   
def classification_report(y_true, y_prediction, type_of_data='Enter Over/Under/Original sampled', type_of_classifier='ClassifierName'):
    """Print Classification report"""
    
    accuracy = accuracy_score(y_true, y_prediction)
    precision = precision_score(y_true, y_prediction)
    recall = recall_score(y_true, y_prediction)
    f1 = f1_score(y_true, y_prediction)
    
    print('Classification Report on Testing Data:\n'+ color.BOLD + type_of_data, 'data\n'+color.END+color.RED+color.BOLD+type_of_classifier,'Classifier'+color.END+color.END)
    print()
    print('---------------------------------------')
    print(color.BOLD + 'Recall: %s' %recall + color.END)
    print('Precision: %s' %precision)
    print('F1 score: %s' %f1)
    print('Accuracy: %s' %accuracy)
    print('---------------------------------------')
    print()


# A function for cross-validation report    
def cross_val_report(classifier, train_data, train_label, cv=10, scoring=['recall','precision', 'f1','accuracy']):
    
    score = cross_validate(classifier, train_data, train_label, cv=cv, scoring= scoring)
    recall = np.mean(score['test_recall'])
    precision = np.mean(score['test_precision'])
    f1 = np.mean(score['test_f1'])
    accuracy= np.mean(score['test_accuracy'])
    print('Cross Validation Report')
    print(color.BOLD + 'Recall: %s' %recall + color.END)
    print('Precision: %s' %precision)
    print('F1: %s' %f1)
    print('Accuracy: %s' %accuracy)
    print()
    print("*Mean values presented")
    print('---------------------------------------')

### Creating the first set of training and test data on imbalanced data

In [ ]:
df = pd.get_dummies(data, columns=['Gender','Occupation'], drop_first = True)
df.head()

#### Creating a model with Original Unbalanced data and measuring metrics

In [ ]:
X_original = df.iloc[:,1:]
y_original = df.iloc[:,0]

In [ ]:
X_original

In [ ]:
X_original.columns

In [ ]:
X_train_orig, X_test_orig, y_train_orig, y_test_orig = train_test_split(X_original,y_original, shuffle = ['True'], stratify=y_original)
X_test_orig.reset_index(inplace = True, drop=True)
y_test_orig.reset_index(inplace = True, drop=True)

### ------------------------------------------------------Start of Model Exploration---------------------------------------------------------------

#### Create undersampled data and fit a model

In [ ]:
X_under_train, y_under_train = NearMiss().fit_resample(X_train_orig, y_train_orig)

In [ ]:
data[data.Target==1].shape

In [ ]:
X_under_train.shape, y_under_train.shape

--------------------

> Model on  an oversampled dataset

In [ ]:
X_over_train, y_over_train = SMOTE().fit_resample(X_train_orig, y_train_orig)

---------------

In [ ]:
print("Original:     "+color.BOLD+ "X_original,y_original"+color.END+"::  X_train_orig, X_test_orig, y_train_orig, y_test_orig")
print()
print("Undersampled:"+color.BOLD+ " X_under, y_under"+color.END+"     ::  X_under_train, y_under_train")
print()
print("Oversampled:"+color.BOLD+ "  X_over, y_over"+color.END+"       ::  X_over_train, y_over_train")

The above datasets can be better sampled by adjusting hyper-parameters of NearMiss and SMOTE, or other methods of sampling could be used

## Random Forest Classifier - Original Data

In [ ]:
clf_rf0 = RandomForestClassifier()
clf_rf0.fit(X_train_orig, y_train_orig)
y_predict= clf_rf0.predict(X_test_orig)
classification_report(y_test_orig, y_predict, 'Original', 'Random Forest')

cross_val_report(clf_rf0, X_under_train, y_under_train)


fig, axs = plt.subplots(1,3, figsize=(15,5))
plot_roc_curve(clf_rf0, X_test_orig, y_test_orig, ax=axs[0])
plot_confusion_matrix(clf_rf0, X_test_orig, y_test_orig, cmap='BuPu', ax=axs[1])

proba = clf_rf0.predict_proba(X_test_orig)[:,1]
axs[2].hist(proba)
axs[2].set_title('Probablity of Target=1')
plt.show()

In [ ]:
confusion_matrix(y_test_orig, y_predict)

In [ ]:
plot_partial_dependence(clf_rf0, X_test_orig, X_test_orig.columns)

-------------

In [ ]:
(pd.Series(clf_rf0.feature_importances_, index=X_test_orig.columns)
   .nlargest(4)
   .plot(kind='bar'))

##### Inferences: 

- The prominent features are: `SCR`,`BALANCE`,`No_OF_CR_TXNS`,`Holding_Period`

## RFE Implementation

In [ ]:
from sklearn.feature_selection import RFE

In [ ]:
rfe = RFE(RandomForestClassifier(), n_features_to_select=4)
rfe = rfe.fit(X_over_train, y_over_train)
# summarize the selection of the attributes
print(rfe.support_)
print(rfe.ranking_)

In [ ]:
X_over_train.columns

##### Inferences:

- RFE algorithm decided prominent features: `SCR`,`BALANCE`,`No_OF_CR_TXNS`,`Holding_Period`

In [ ]:
categorical_features_indices = np.where(X_train_orig.dtypes != np.float)[0]

In [ ]:
categorical_features_indices

## RandomForest Classifier - Undersampled Data

In [ ]:
X_train_orig.columns

In [ ]:
clf_rf1 = RandomForestClassifier()
clf_rf1.fit(X_under_train, y_under_train)
y_predict= clf_rf1.predict(X_test_orig)
classification_report(y_test_orig, y_predict, 'Undersampled', 'Random Forest')
cross_val_report(clf_rf1, X_under_train, y_under_train)

fig, axs = plt.subplots(1,3, figsize=(15,5))
plot_roc_curve(clf_rf1, X_test_orig, y_test_orig, ax=axs[0])
plot_confusion_matrix(clf_rf1, X_test_orig, y_test_orig, cmap='BuPu', ax=axs[1])

proba = clf_rf1.predict_proba(X_test_orig)[:,1]
axs[2].hist(proba)
axs[2].set_title('Probablity of Target=1')
plt.show()

In [ ]:
confusion_matrix(y_test_orig, y_predict)

### Rank Ordering Stats for Random Forest - Undersampled Data

In [ ]:
proba_df = pd.DataFrame(proba, columns=['Probablity'])
proba_df['present_decile'] = pd.qcut(proba_df.Probablity, 10, labels=[str(i+1) for i in range(10)])
proba_df.present_decile = proba_df.present_decile.astype(int)

proba_df['response_test'] = y_test_orig
proba_df['response_model'] = proba_df['Probablity'].apply(lambda x: 0 if x<=0.5 else 1)
proba_df['correct_loan_prediction'] = 0
for ind, row in proba_df.iterrows():
    if row['response_test'] == row['response_model'] == 1:
        proba_df.loc[ind, 'correct_loan_prediction'] =1


ks_table = pd.DataFrame({'Decile':[j for j in range(10,0,-1)]})

ks_table['base'] =0
ks_table['responses'] =0
ks_table['non_responses'] =0
for ind, row in ks_table.iterrows():
    ks_table.loc[ind, 'base'] = proba_df[proba_df.present_decile==row['Decile']].shape[0]
    
    try:
        correct_loan_prediction1 = proba_df[proba_df.present_decile==row['Decile']].correct_loan_prediction.value_counts()[1]
    except:
        correct_loan_prediction1 = 0
        
    ks_table.loc[ind, 'responses'] = correct_loan_prediction1
    
ks_table['non_responses'] = ks_table['base'] - ks_table['responses']
    
ks_table['resp_rate'] = ks_table['responses']/ks_table['base']*100
ks_table['non_resp_rate'] = ks_table['non_responses']/ks_table['base']*100

ks_table['cum_base'] = ks_table.base.cumsum()
ks_table['cum_resp'] = ks_table.responses.cumsum()
ks_table['cum_non_resp'] = ks_table.non_responses.cumsum()

ks_table['cum_resp%'] = ks_table['cum_resp']/ks_table['cum_resp'].max()*100
ks_table['cum_non_resp%'] = ks_table['cum_non_resp']/ks_table['cum_non_resp'].max()*100
ks_table['ks'] = ks_table['cum_resp%']-ks_table['cum_non_resp%']

ks_table['na_label'] = ks_table['cum_resp']/ks_table['cum_base']*100
ks_table['lift'] = ks_table['na_label']/(100*ks_table['cum_resp'].max()/ks_table['cum_base'].max())

ks_table

### Gain Chart

In [ ]:
plt.plot([i for i in range(1,11)],ks_table['cum_resp%'], label = 'Targeted Response')
plt.plot([i for i in range(1,11)], np.arange(0, 110, 10)[1:], label = 'Random approach')

plt.xticks(np.arange(1, 11), labels=[(str(i)+'0%') for i in list(range(0,11))])
plt.yticks(np.arange(0, 110, 10), labels=[(str(i)+'0%') for i in list(range(0,11))])

plt.title('Gain Chart')
plt.ylabel('Cumulative Percentage of Responders Captured')
plt.xlabel('Population')
plt.legend()
plt.grid()
plt.show()

### Lift Chart

In [ ]:
plt.plot([i for i in range(1,11)],ks_table.lift*100, label = 'Targeted Response')
plt.plot([i for i in range(1,11)], [100, 100, 100, 100, 100, 100, 100, 100, 100, 100], label = 'Random approach')

plt.xticks(np.arange(1, 11), labels=[(str(i)) for i in list(range(1,11))])
plt.yticks(np.arange(0, 900, 100), labels=[(str(i)+'00%') for i in list(range(0,11))])

plt.title('Lift Chart')
plt.ylabel('Lift%')
plt.xlabel('Decile')
plt.legend()
plt.grid()

plt.show()

### KS chart

In [ ]:
plt.plot([i for i in range(1,11)],ks_table['cum_resp%'], label = 'Positive Response')
plt.plot([i for i in range(1,11)],ks_table['cum_non_resp%'], label = 'Negative Response')
# plt.plot([i for i in range(1,11)], np.arange(0, 110, 10)[1:], label = 'Negative')

plt.xticks(np.arange(1, 11), labels=[(str(i)) for i in list(range(1,11))])
plt.yticks(np.arange(0, 110, 10), labels=[(str(i)+'0%') for i in list(range(0,11))])

plt.title('KS chart')
plt.ylabel('Cumulative Responses')
plt.xlabel('Deciles')
plt.legend()
plt.grid()
plt.show()

-------------

## RandomForest Classifier - Oversampled Data

In [ ]:
clf_rf2 = RandomForestClassifier()

In [ ]:
clf_rf2.fit(X_over_train, y_over_train)
y_predict= clf_rf2.predict(X_test_orig)
classification_report(y_test_orig, y_predict, 'Oversampled', 'Random Forest')

cross_val_report(clf_rf2, X_over_train, y_over_train)



fig, axs = plt.subplots(1,3, figsize=(15,5))
plot_roc_curve(clf_rf2, X_test_orig, y_test_orig, ax=axs[0])
plot_confusion_matrix(clf_rf2, X_test_orig, y_test_orig, cmap='BuPu', ax=axs[1])

proba = clf_rf2.predict_proba(X_test_orig)[:,1]
axs[2].hist(proba)
axs[2].set_title('Probablity of Target=1')
plt.show()

In [ ]:
confusion_matrix(y_test_orig, y_predict)

In [ ]:
(pd.Series(clf_rf2.feature_importances_, index=X_test_orig.columns)
   .nlargest(4)
   .plot(kind='barh'))

### Rank Ordering Stats for Random Forest - Oversampled Data

In [ ]:
proba_df = pd.DataFrame(proba, columns=['Probablity'])
proba_df['present_decile'] = pd.qcut(proba_df.Probablity, 10, labels=[str(i+1) for i in range(10)])
proba_df.present_decile = proba_df.present_decile.astype(int)

proba_df['response_test'] = y_test_orig
proba_df['response_model'] = proba_df['Probablity'].apply(lambda x: 0 if x<=0.5 else 1)
proba_df['correct_loan_prediction'] = 0
for ind, row in proba_df.iterrows():
    if row['response_test'] == row['response_model'] == 1:
        proba_df.loc[ind, 'correct_loan_prediction'] =1


ks_table = pd.DataFrame({'Decile':[j for j in range(10,0,-1)]})

ks_table['base'] =0
ks_table['responses'] =0
ks_table['non_responses'] =0
for ind, row in ks_table.iterrows():
    ks_table.loc[ind, 'base'] = proba_df[proba_df.present_decile==row['Decile']].shape[0]
    
    try:
        correct_loan_prediction1 = proba_df[proba_df.present_decile==row['Decile']].correct_loan_prediction.value_counts()[1]
    except:
        correct_loan_prediction1 = 0
        
    ks_table.loc[ind, 'responses'] = correct_loan_prediction1
    
ks_table['non_responses'] = ks_table['base'] - ks_table['responses']
    
ks_table['resp_rate'] = ks_table['responses']/ks_table['base']*100
ks_table['non_resp_rate'] = ks_table['non_responses']/ks_table['base']*100

ks_table['cum_base'] = ks_table.base.cumsum()
ks_table['cum_resp'] = ks_table.responses.cumsum()
ks_table['cum_non_resp'] = ks_table.non_responses.cumsum()

ks_table['cum_resp%'] = ks_table['cum_resp']/ks_table['cum_resp'].max()*100
ks_table['cum_non_resp%'] = ks_table['cum_non_resp']/ks_table['cum_non_resp'].max()*100
ks_table['ks'] = ks_table['cum_resp%']-ks_table['cum_non_resp%']

ks_table['na_label'] = ks_table['cum_resp']/ks_table['cum_base']*100
ks_table['lift'] = ks_table['na_label']/(100*ks_table['cum_resp'].max()/ks_table['cum_base'].max())

ks_table

### Gain Chart

In [ ]:
plt.plot([i for i in range(1,11)],ks_table['cum_resp%'], label = 'Targeted Response')
plt.plot([i for i in range(1,11)], np.arange(0, 110, 10)[1:], label = 'Random approach')

plt.xticks(np.arange(1, 11), labels=[(str(i)+'0%') for i in list(range(0,11))])
plt.yticks(np.arange(0, 110, 10), labels=[(str(i)+'0%') for i in list(range(0,11))])

plt.title('Gain Chart')
plt.ylabel('Cumulative Percentage of Responders Captured')
plt.xlabel('Population')
plt.legend()
plt.grid()
plt.show()

### Lift Chart

In [ ]:
plt.plot([i for i in range(1,11)],ks_table.lift*100, label = 'Targeted Response')
plt.plot([i for i in range(1,11)], [100, 100, 100, 100, 100, 100, 100, 100, 100, 100], label = 'Random approach')

plt.xticks(np.arange(1, 11), labels=[(str(i)) for i in list(range(1,11))])
plt.yticks(np.arange(0, 900, 100), labels=[(str(i)+'00%') for i in list(range(0,11))])

plt.title('Lift Chart')
plt.ylabel('Lift%')
plt.xlabel('Decile')
plt.legend()
plt.grid()

plt.show()

### KS chart

In [ ]:
plt.plot([i for i in range(1,11)],ks_table['cum_resp%'], label = 'Positive Response')
plt.plot([i for i in range(1,11)],ks_table['cum_non_resp%'], label = 'Negative Response')
# plt.plot([i for i in range(1,11)], np.arange(0, 110, 10)[1:], label = 'Negative')

plt.xticks(np.arange(1, 11), labels=[(str(i)) for i in list(range(1,11))])
plt.yticks(np.arange(0, 110, 10), labels=[(str(i)+'0%') for i in list(range(0,11))])

plt.title('KS chart')
plt.ylabel('Cumulative Responses')
plt.xlabel('Deciles')
plt.legend()
plt.grid()
plt.show()

----------
----------------

## Logistic Regression - Original Data

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr0 = LogisticRegression(max_iter=1000)
lr0.fit(X_train_orig, y_train_orig)
y_predict= lr0.predict(X_test_orig)
classification_report(y_test_orig, y_predict, 'Original', 'Logistic Regression')

cross_val_report(lr0, X_under_train, y_under_train)

fig, axs = plt.subplots(1,3, figsize=(15,5))
plot_roc_curve(lr0, X_test_orig, y_test_orig, ax=axs[0])
plot_confusion_matrix(lr0, X_test_orig, y_test_orig, cmap='BuPu', ax=axs[1])

proba = lr0.predict_proba(X_test_orig)[:,1]
axs[2].hist(proba)
axs[2].set_title('Probablity of Target=1')
plt.show()

In [ ]:
confusion_matrix(y_test_orig, y_predict)

### Rank Ordering Stats for Logistic Regression - Original Data

In [ ]:
proba_df = pd.DataFrame(proba, columns=['Probablity'])
proba_df['present_decile'] = pd.qcut(proba_df.Probablity, 10, labels=[str(i+1) for i in range(10)])
proba_df.present_decile = proba_df.present_decile.astype(int)

proba_df['response_test'] = y_test_orig
proba_df['response_model'] = proba_df['Probablity'].apply(lambda x: 0 if x<=0.5 else 1)
proba_df['correct_loan_prediction'] = 0
for ind, row in proba_df.iterrows():
    if row['response_test'] == row['response_model'] == 1:
        proba_df.loc[ind, 'correct_loan_prediction'] =1


ks_table = pd.DataFrame({'Decile':[j for j in range(10,0,-1)]})

ks_table['base'] =0
ks_table['responses'] =0
ks_table['non_responses'] =0
for ind, row in ks_table.iterrows():
    ks_table.loc[ind, 'base'] = proba_df[proba_df.present_decile==row['Decile']].shape[0]
    
    try:
        correct_loan_prediction1 = proba_df[proba_df.present_decile==row['Decile']].correct_loan_prediction.value_counts()[1]
    except:
        correct_loan_prediction1 = 0
        
    ks_table.loc[ind, 'responses'] = correct_loan_prediction1
    
ks_table['non_responses'] = ks_table['base'] - ks_table['responses']
    
ks_table['resp_rate'] = ks_table['responses']/ks_table['base']*100
ks_table['non_resp_rate'] = ks_table['non_responses']/ks_table['base']*100

ks_table['cum_base'] = ks_table.base.cumsum()
ks_table['cum_resp'] = ks_table.responses.cumsum()
ks_table['cum_non_resp'] = ks_table.non_responses.cumsum()

ks_table['cum_resp%'] = ks_table['cum_resp']/ks_table['cum_resp'].max()*100
ks_table['cum_non_resp%'] = ks_table['cum_non_resp']/ks_table['cum_non_resp'].max()*100
ks_table['ks'] = ks_table['cum_resp%']-ks_table['cum_non_resp%']

ks_table['na_label'] = ks_table['cum_resp']/ks_table['cum_base']*100
ks_table['lift'] = ks_table['na_label']/(100*ks_table['cum_resp'].max()/ks_table['cum_base'].max())

ks_table

### Gain Chart

In [ ]:
plt.plot([i for i in range(1,11)],ks_table['cum_resp%'], label = 'Targeted Response')
plt.plot([i for i in range(1,11)], np.arange(0, 110, 10)[1:], label = 'Random approach')

plt.xticks(np.arange(1, 11), labels=[(str(i)+'0%') for i in list(range(0,11))])
plt.yticks(np.arange(0, 110, 10), labels=[(str(i)+'0%') for i in list(range(0,11))])

plt.title('Gain Chart')
plt.ylabel('Cumulative Percentage of Responders Captured')
plt.xlabel('Population')
plt.legend()
plt.grid()
plt.show()

### Lift Chart

In [ ]:
plt.plot([i for i in range(1,11)],ks_table.lift*100, label = 'Targeted Response')
plt.plot([i for i in range(1,11)], [100, 100, 100, 100, 100, 100, 100, 100, 100, 100], label = 'Random approach')

plt.xticks(np.arange(1, 11), labels=[(str(i)) for i in list(range(1,11))])
plt.yticks(np.arange(0, 900, 100), labels=[(str(i)+'00%') for i in list(range(0,11))])

plt.title('Lift Chart')
plt.ylabel('Lift%')
plt.xlabel('Decile')
plt.legend()
plt.grid()

plt.show()

### KS chart

In [ ]:
plt.plot([i for i in range(1,11)],ks_table['cum_resp%'], label = 'Positive Response')
plt.plot([i for i in range(1,11)],ks_table['cum_non_resp%'], label = 'Negative Response')
# plt.plot([i for i in range(1,11)], np.arange(0, 110, 10)[1:], label = 'Negative')

plt.xticks(np.arange(1, 11), labels=[(str(i)) for i in list(range(1,11))])
plt.yticks(np.arange(0, 110, 10), labels=[(str(i)+'0%') for i in list(range(0,11))])

plt.title('KS chart')
plt.ylabel('Cumulative Responses')
plt.xlabel('Deciles')
plt.legend()
plt.grid()
plt.show()

## Logistic Regression - Undersampled Data

In [ ]:
lr1 = LogisticRegression(max_iter=100)
lr1.fit(X_under_train, y_under_train)
y_predict= lr1.predict(X_test_orig)
classification_report(y_test_orig, y_predict, 'Undersampled', 'Logistic Regression')

cross_val_report(lr1, X_under_train, y_under_train)

fig, axs = plt.subplots(1,3, figsize=(15,5))
plot_roc_curve(lr1, X_test_orig, y_test_orig, ax=axs[0])
plot_confusion_matrix(lr1, X_test_orig, y_test_orig, cmap='BuPu', ax=axs[1])

proba = lr1.predict_proba(X_test_orig)[:,1]
axs[2].hist(proba)
axs[2].set_title('Probablity of Target=1')
plt.show()

In [ ]:
confusion_matrix(y_test_orig, y_predict)

### Rank Ordering Stats for Logistic Regression - Undersampled Data

In [ ]:
proba_df = pd.DataFrame(proba, columns=['Probablity'])
proba_df['present_decile'] = pd.qcut(proba_df.Probablity, 10, labels=[str(i+1) for i in range(10)])
proba_df.present_decile = proba_df.present_decile.astype(int)

proba_df['response_test'] = y_test_orig
proba_df['response_model'] = proba_df['Probablity'].apply(lambda x: 0 if x<=0.5 else 1)
proba_df['correct_loan_prediction'] = 0
for ind, row in proba_df.iterrows():
    if row['response_test'] == row['response_model'] == 1:
        proba_df.loc[ind, 'correct_loan_prediction'] =1


ks_table = pd.DataFrame({'Decile':[j for j in range(10,0,-1)]})

ks_table['base'] =0
ks_table['responses'] =0
ks_table['non_responses'] =0
for ind, row in ks_table.iterrows():
    ks_table.loc[ind, 'base'] = proba_df[proba_df.present_decile==row['Decile']].shape[0]
    
    try:
        correct_loan_prediction1 = proba_df[proba_df.present_decile==row['Decile']].correct_loan_prediction.value_counts()[1]
    except:
        correct_loan_prediction1 = 0
        
    ks_table.loc[ind, 'responses'] = correct_loan_prediction1
    
ks_table['non_responses'] = ks_table['base'] - ks_table['responses']
    
ks_table['resp_rate'] = ks_table['responses']/ks_table['base']*100
ks_table['non_resp_rate'] = ks_table['non_responses']/ks_table['base']*100

ks_table['cum_base'] = ks_table.base.cumsum()
ks_table['cum_resp'] = ks_table.responses.cumsum()
ks_table['cum_non_resp'] = ks_table.non_responses.cumsum()

ks_table['cum_resp%'] = ks_table['cum_resp']/ks_table['cum_resp'].max()*100
ks_table['cum_non_resp%'] = ks_table['cum_non_resp']/ks_table['cum_non_resp'].max()*100
ks_table['ks'] = ks_table['cum_resp%']-ks_table['cum_non_resp%']

ks_table['na_label'] = ks_table['cum_resp']/ks_table['cum_base']*100
ks_table['lift'] = ks_table['na_label']/(100*ks_table['cum_resp'].max()/ks_table['cum_base'].max())

ks_table

### Gain Chart

In [ ]:
plt.plot([i for i in range(1,11)],ks_table['cum_resp%'], label = 'Targeted Response')
plt.plot([i for i in range(1,11)], np.arange(0, 110, 10)[1:], label = 'Random approach')

plt.xticks(np.arange(1, 11), labels=[(str(i)+'0%') for i in list(range(0,11))])
plt.yticks(np.arange(0, 110, 10), labels=[(str(i)+'0%') for i in list(range(0,11))])

plt.title('Gain Chart')
plt.ylabel('Cumulative Percentage of Responders Captured')
plt.xlabel('Population')
plt.legend()
plt.grid()
plt.show()

### Lift Chart

In [ ]:
plt.plot([i for i in range(1,11)],ks_table.lift*100, label = 'Targeted Response')
plt.plot([i for i in range(1,11)], [100, 100, 100, 100, 100, 100, 100, 100, 100, 100], label = 'Random approach')

plt.xticks(np.arange(1, 11), labels=[(str(i)) for i in list(range(1,11))])
plt.yticks(np.arange(0, 900, 100), labels=[(str(i)+'00%') for i in list(range(0,11))])

plt.title('Lift Chart')
plt.ylabel('Lift%')
plt.xlabel('Decile')
plt.legend()
plt.grid()

plt.show()

### KS chart

In [ ]:
plt.plot([i for i in range(1,11)],ks_table['cum_resp%'], label = 'Positive Response')
plt.plot([i for i in range(1,11)],ks_table['cum_non_resp%'], label = 'Negative Response')
# plt.plot([i for i in range(1,11)], np.arange(0, 110, 10)[1:], label = 'Negative')

plt.xticks(np.arange(1, 11), labels=[(str(i)) for i in list(range(1,11))])
plt.yticks(np.arange(0, 110, 10), labels=[(str(i)+'0%') for i in list(range(0,11))])

plt.title('KS chart')
plt.ylabel('Cumulative Responses')
plt.xlabel('Deciles')
plt.legend()
plt.grid()
plt.show()

## Logistic Regression - Oversampled Data

In [ ]:
lr2 = LogisticRegression(max_iter=500)
lr2.fit(X_over_train, y_over_train)
y_predict= lr2.predict(X_test_orig)
classification_report(y_test_orig, y_predict, 'Oversampled', 'Logistic Regression')

cross_val_report(lr2, X_over_train, y_over_train)

fig, axs = plt.subplots(1,3, figsize=(15,5))
plot_roc_curve(lr2, X_test_orig, y_test_orig, ax=axs[0])
plot_confusion_matrix(lr2, X_test_orig, y_test_orig, cmap='BuPu', ax=axs[1])

proba = lr2.predict_proba(X_test_orig)[:,1]
axs[2].hist(proba)
axs[2].set_title('Probablity of Target=1')
plt.show()

In [ ]:
confusion_matrix(y_test_orig, y_predict)

### Rank Ordering Stats for Logistic Regression - Oversampled Data

In [ ]:
proba_df = pd.DataFrame(proba, columns=['Probablity'])
proba_df['present_decile'] = pd.qcut(proba_df.Probablity, 10, labels=[str(i+1) for i in range(10)])
proba_df.present_decile = proba_df.present_decile.astype(int)

proba_df['response_test'] = y_test_orig
proba_df['response_model'] = proba_df['Probablity'].apply(lambda x: 0 if x<=0.5 else 1)
proba_df['correct_loan_prediction'] = 0
for ind, row in proba_df.iterrows():
    if row['response_test'] == row['response_model'] == 1:
        proba_df.loc[ind, 'correct_loan_prediction'] =1


ks_table = pd.DataFrame({'Decile':[j for j in range(10,0,-1)]})

ks_table['base'] =0
ks_table['responses'] =0
ks_table['non_responses'] =0
for ind, row in ks_table.iterrows():
    ks_table.loc[ind, 'base'] = proba_df[proba_df.present_decile==row['Decile']].shape[0]
    
    try:
        correct_loan_prediction1 = proba_df[proba_df.present_decile==row['Decile']].correct_loan_prediction.value_counts()[1]
    except:
        correct_loan_prediction1 = 0
        
    ks_table.loc[ind, 'responses'] = correct_loan_prediction1
    
ks_table['non_responses'] = ks_table['base'] - ks_table['responses']
    
ks_table['resp_rate'] = ks_table['responses']/ks_table['base']*100
ks_table['non_resp_rate'] = ks_table['non_responses']/ks_table['base']*100

ks_table['cum_base'] = ks_table.base.cumsum()
ks_table['cum_resp'] = ks_table.responses.cumsum()
ks_table['cum_non_resp'] = ks_table.non_responses.cumsum()

ks_table['cum_resp%'] = ks_table['cum_resp']/ks_table['cum_resp'].max()*100
ks_table['cum_non_resp%'] = ks_table['cum_non_resp']/ks_table['cum_non_resp'].max()*100
ks_table['ks'] = ks_table['cum_resp%']-ks_table['cum_non_resp%']

ks_table['na_label'] = ks_table['cum_resp']/ks_table['cum_base']*100
ks_table['lift'] = ks_table['na_label']/(100*ks_table['cum_resp'].max()/ks_table['cum_base'].max())

ks_table

### Gain Chart

In [ ]:
plt.plot([i for i in range(1,11)],ks_table['cum_resp%'], label = 'Targeted Response')
plt.plot([i for i in range(1,11)], np.arange(0, 110, 10)[1:], label = 'Random approach')

plt.xticks(np.arange(1, 11), labels=[(str(i)+'0%') for i in list(range(0,11))])
plt.yticks(np.arange(0, 110, 10), labels=[(str(i)+'0%') for i in list(range(0,11))])

plt.title('Gain Chart')
plt.ylabel('Cumulative Percentage of Responders Captured')
plt.xlabel('Population')
plt.legend()
plt.grid()
plt.show()

### Lift Chart

In [ ]:
plt.plot([i for i in range(1,11)],ks_table.lift*100, label = 'Targeted Response')
plt.plot([i for i in range(1,11)], [100, 100, 100, 100, 100, 100, 100, 100, 100, 100], label = 'Random approach')

plt.xticks(np.arange(1, 11), labels=[(str(i)) for i in list(range(1,11))])
plt.yticks(np.arange(0, 900, 100), labels=[(str(i)+'00%') for i in list(range(0,11))])

plt.title('Lift Chart')
plt.ylabel('Lift%')
plt.xlabel('Decile')
plt.legend()
plt.grid()

plt.show()

### KS chart

In [ ]:
plt.plot([i for i in range(1,11)],ks_table['cum_resp%'], label = 'Positive Response')
plt.plot([i for i in range(1,11)],ks_table['cum_non_resp%'], label = 'Negative Response')
# plt.plot([i for i in range(1,11)], np.arange(0, 110, 10)[1:], label = 'Negative')

plt.xticks(np.arange(1, 11), labels=[(str(i)) for i in list(range(1,11))])
plt.yticks(np.arange(0, 110, 10), labels=[(str(i)+'0%') for i in list(range(0,11))])

plt.title('KS chart')
plt.ylabel('Cumulative Responses')
plt.xlabel('Deciles')
plt.legend()
plt.grid()
plt.show()

------
------

### ------------------------------------------------------End of Model Exploration---------------------------------------------------------------

## PRE-BUCKETING VISUALIZATIONS FOR INFERENCES

### SCR Vs Occupation Vs Target

In [ ]:
sns.catplot(x="Occupation", y="SCR", hue = "Target",data=data, kind = "violin", split = True)
plt.show()

##### Inferences: 

- `SELF-EMP` class tends to have more `SCR` values who are likely to opt for loans.
- `SAL` class shows uniformity across `SCR` for people opting/not opting for loans.
- `SENP` business class tend to show similar fashion like `SELF-EMP` class when it comes to opting for loans.
- `PROF` class tends to lie somewhere between `SENP` and `SAL` class.

### SCR Vs Occupation Vs Target

In [ ]:
sns.relplot(x="Occupation", y="SCR", hue = "Target",data=data, aspect=1.5, kind = "line")
plt.show()

### SCR Vs Occupation Vs Target

In [ ]:
sns.catplot(x="Occupation", y="SCR", hue = "Target",data=data, aspect=2.0, kind = "point")
plt.show()

##### Inferences: 

- `SELF-EMP`class, there can be seen a high `SCR` value ranging between 660 to 700 points who have shown interest in opting for loans whereas least interest in the same class was shown at 540 to 550 points of `SCR`

- `SENP` class. are the second in line to opt for Loans, as they tend to show comaratively hgiher `SCR` values in the line graph above.

In [ ]:
data1 = data.copy()

## Bucketing for AGE values

#### pd.qcut() function

In [ ]:
data1['Ageb'] = pd.qcut(data['Age'], q=4)

In [ ]:
data1.head()

#### AGE Intervals

In [ ]:
data1.Ageb.value_counts()

In [ ]:
data1.dtypes

In [ ]:
data1.Ageb

In [ ]:
data1.head(5)

In [ ]:
# sns.pairplot(data1, hue='Target')

In [ ]:
data1 = pd.get_dummies(data1)

In [ ]:
data1.head(2)

In [ ]:
data2 = data1.copy()

In [ ]:
data1.columns

### Age Bucket Ranking Function

In [ ]:
cols = ['Ageb_(30.0, 38.0]', 'Ageb_(20.999, 30.0]','Ageb_(46.0, 55.0]','Ageb_(38.0, 46.0]']

def func1(x):
    if x ==1:
        return 1
def func2(x):
    if x ==1:
        return 2

def func3(x):
    if x ==1:
        return 3

def func4(x):
    if x ==1:
        return 4


data1['age_d'] = data1['Ageb_(30.0, 38.0]'].apply(func1)
data1['age_d'] = data1['Ageb_(20.999, 30.0]'].apply(func2)
data1['age_d'] = data1['Ageb_(46.0, 55.0]'].apply(func3)
data1['age_d'] = data1['Ageb_(38.0, 46.0]'].apply(func4)

In [ ]:
for ind, row in data1.iterrows():
    if row['Ageb_(20.999, 30.0]'] ==1:
        data1.loc[ind, 'age_d'] = 1
    elif row['Ageb_(30.0, 38.0]'] ==1:
        data1.loc[ind, 'age_d'] = 2
    elif row['Ageb_(38.0, 46.0]'] ==1:
        data1.loc[ind, 'age_d'] = 3
    elif row['Ageb_(46.0, 55.0]'] ==1:
        data1.loc[ind, 'age_d'] = 4

In [ ]:
data1.age_d.value_counts()

#### Converting Age buckets to integer type

In [ ]:
data1.age_d = data1.age_d.astype('int32')

In [ ]:
data1.head()

## POST-BUCKETING VISUALIZATIONS FOR INFERENCES

### SCR Vs Age-Category Vs Target

AGE Intervals & Ranking

- (20.9, 30.0]   ----> 1
- (30.0, 38.0]   ----> 2
- (38.0, 46.0]   ----> 3
- (46.0, 55.0]   ----> 4

In [ ]:
sns.catplot(x="age_d", y="SCR", hue = "Target",data=data1, kind = "violin", split = True)
plt.show()

In [ ]:
sns.catplot(x="age_d", y="SCR", hue = "Target",data=data1, aspect=2.0, kind = "point")
plt.show()

##### Inferences: 

- The `Age groups` `(20.9-30] years` and `(38-46] years` tend to showwcase higher `SCR` values hence more interest towards opting for lonans. However, the age groups (30-38] and (46.0-55) showcase least interest in opting for loans.

### SCR Vs Age-Category Vs Target

In [ ]:
sns.catplot(x="age_d", y="SCR", hue = "Target",data=data1, kind = "swarm")
plt.show()

In [ ]:
sns.set_style('ticks')

## BUCKETING SCR & HOLDING PERIODS

In [ ]:
data1['SCRb'] = pd.qcut(data['SCR'], q=5)

In [ ]:
data1.SCRb.value_counts()

In [ ]:
data1['HPB'] = pd.qcut(data['Holding_Period'], q=4)

In [ ]:
data1.HPB.value_counts()

In [ ]:
data1.head()

### SCR-Bucketed Vs Age-Category Vs Target

In [ ]:
sns.catplot(x="SCRb", y="age_d", hue = "Target",data=data1, kind = "point", split = True,)
plt.show()
plt.figure(figsize=(10,10))

In [ ]:
# sns.catplot(x="age_d", y="SCRb", hue = "Target",data=data1, kind = "swarm")
# plt.show()

In [ ]:
sns.catplot(x="SCRb", y="age_d", hue = "Target",data=data1, kind = "violin", split = True)
plt.show()

##### Inferences: 

- People in the `Age Group` `(46.0, 55.0]` years tend to have higher `SCR` score i.e. 826 to 1000 points and opt for loans as compared to `(30.0, 38.0]` years who stand at the second position for higher `SCR` values i.e. 650 to 830 points.

- People in the `Age Group` `(38.0, 46.0]` years tend to have `SCR` values between 400 to 650 points.

### Holding Period Bucketed Vs Age-Category Vs Target

In [ ]:
sns.catplot(x="HPB", y="age_d", hue = "Target",data=data1, kind = "violin", split = True)
plt.show()

##### Inferences: 

- Higher the `Holding Period` more stable is the money in the person's account. 
- `Holding Period` of 23-31 weeks tends to be uniform across all the `Age Groups`
- `Holding Period` between 1-8 weeks tends to lie in the `Age Groups` = 2 and 3 i.e. (30.0, 38.0] and (38.0, 46.0]

In [ ]:
sns.catplot(x="HPB", y="age_d", hue = "Target",data=data1, kind = "point")
plt.show()

### Pair-Plot on Bucketed Data

In [ ]:
# sns.pairplot(data1, hue = 'Target')
# plt.show()

##### Inferences from Pair-Plot: 

- People who tend to opt for loan have less `BALANCE` w.r.t people not buying loan.
- People having higher `BALANCE` tend to be in between 35-50 years of `AGE`.
- People having less `BALANCE` tend to show more interest towards loans i.e. higher `SCR` values.
- People having less `BALANCE` tend to have higher `No of Credit transactions` opt for loans.
- People having less `BALANCE` have higher `HOLDING PERIOD` who opt for loans.

------

### ----------------------------------------------------Start of Black Box Models Exploration------------------------------------------------

## SHAP - BlackBox Model

In [ ]:
# import shap

In [ ]:
# X_train_summary = shap.kmeans(X_train_orig, 10)

In [ ]:
# explainerKNN = shap.KernelExplainer(clf_KNN0.predict,X_train_summary)

In [ ]:
# shap_values_KNN_test = explainerKNN.shap_values(X_test_orig)

In [ ]:
# shap.initjs()
# shap.force_plot(explainerKNN.expected_value, shap_values_KNN_test[:1000,:], X_test_orig.iloc[:1000,:])

## LIME- BlackBox Model

In [ ]:
# import lime
# import lime.lime_tabular

In [ ]:
# lgb_params = {
#   'task': 'train',
#     'boosting_type': 'goss',
#     'objective': 'binary',
#     'metric':'binary_logloss',
#     'metric': {'l2', 'auc'},
#     'num_leaves': 50,
#     'learning_rate': 0.1,
#     'feature_fraction': 0.8,
#     'bagging_fraction': 0.8,
#     'verbose': None,
#     'num_iteration':100,
#     'num_threads':7,
#     'max_depth':12,
#     'min_data_in_leaf':100,
#     'alpha':0.5}

In [ ]:
# import warnings
# from lime import submodular_pick
# import lightgbm as lgb

# lgb_train = lgb.Dataset(X_train_orig, y_train_orig)
# lgb_eval = lgb.Dataset(X_test_orig, y_test_orig)

# model = lgb.train(lgb_params,lgb_train,num_boost_round=20,valid_sets=lgb_eval,early_stopping_rounds=5)
# lime.lime_tabular.LimeTabularExplainer(data[model.feature_name()].astype(int).values,  
# mode='classification',training_labels=data3['Target'],feature_names=model.feature_name())
# # Remember to convert the dataframe to matrix values
# # SP-LIME returns exaplanations on a sample set to provide a non redundant global decision boundary of original model
# sp_obj = submodular_pick.SubmodularPick(explainer,data3[model.feature_name()].values, \
# prob, num_features=5,num_exps_desired=10)

# [exp.as_pyplot_figure(label=1) for exp in sp_obj.sp_explanations]

## CatBoost Classifier

In [ ]:
# from catboost import CatBoostClassifier
# model = CatBoostClassifier(random_seed=42)

In [ ]:
# cate_features_index = np.where(data.dtypes != float)[0]

In [ ]:
# X_over_train

In [ ]:
# model.fit(X_under_train,y_under_train,cat_features=cate_features_index)

In [ ]:
# test = pd.read_csv("test_data.csv")
# pred = model.predict(test)


In [ ]:
# from sklearn.metrics import confusion_matrix

In [ ]:
# confusion_matrix(y_test_orig, pred)

### ----------------------------------------------------End of Black Box Models Exploration------------------------------------------------

-----

## Model Evaluation Matrix - Which Models to be Considered?

In [ ]:
model_eval = {'Good Fit Percent%': ['10.9','9.84','19.6','17.2','8.6','18.4','11.4','25.0','25.0','16.4','20.8','17.2'],
        'Comments': ['Not Considering','Not Considering','Not Considering','Not Considering','Not Considering','Not Considering',
                     'Not Considering','Considering','Considering','Not Considering','Considering','Not Considering',]
        }

df1 = pd.DataFrame(model_eval, columns = ['Good Fit Percent%','Comments'], index=['DecisionTreeClassifier - Original Data','DecisionTreeClassifier - UnderSampled Data','DecisionTreeClassifier - OverSampled Data',
                                                                           'KNN Classifier - Original Data','KNN Classifier - Undersampled Data','Random Forest Classifier - Original Data','RandomForest Classifier - Undersampled Data',
                                                                           'RandomForest Classifier - Oversampled Data','Logistic Regression - Original Data','Logistic Regression - Undersampled Data',
                                                                          'Logistic Regression - Oversampled Data','KNN Classifier - Oversampled Data'])

print (df1)

In [ ]:
data1.to_excel("F:\\IBPA_IIM_FinalProject\\data1.xlsx")

In [ ]:
df1 = data._get_numeric_data()

In [ ]:
df1

In [ ]:
df1.head()

In [ ]:
vif = pd.DataFrame()

In [ ]:
vif["VIF Factor"] = [variance_inflation_factor(X_original.values, i) for i in range(X_original.shape[1])]

In [ ]:
vif["features"] = X_original.columns

In [ ]:
vif.round(1)

In [ ]:
vif.round(3)

In [ ]:
from matplotlib import pyplot as plt
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier 
from sklearn import tree



In [ ]:
clf = DecisionTreeClassifier(max_depth = 5)
clf.fit(X_train_orig, y_train_orig)
y_prediction_orig = clf.predict(X_test_orig)
classification_report(y_test_orig, y_prediction_orig, 'Original', 'Decision Tree')
plot_roc_curve(clf, X_test_orig, y_test_orig)
plt.show()
# cross_val_report(clf, y_test_orig,y_under_prediction.reshape(1,-1))

In [ ]:
clf_o = DecisionTreeClassifier(max_depth = 5)
clf_o.fit(X_over_train, y_over_train)
y_prediction_over = clf_o.predict(X_over_train)
classification_report(y_over_test, y_prediction_over, 'OverSampled', 'Decision Tree')
plot_roc_curve(clf_o, X_over_test, y_over_test)
plt.show()
# cross_val_report(clf, y_test_orig,y_under_prediction.reshape(1,-1))

In [ ]:
clf_u = DecisionTreeClassifier(max_depth = 5)
clf_u.fit(X_under_train, y_under_train)
y_prediction_under = clf_u.predict(X_under_train)
classification_report(y_under_test, y_prediction_under, 'UnderSampled', 'Decision Tree')
plot_roc_curve(clf_u, X_under_test, y_under_test)
plt.show()
# cross_val_report(clf, y_test_orig,y_under_prediction.reshape(1,-1))

In [ ]:
# fig = plt.figure(figsize=(50,20), dpi = 800)
# tree.plot_tree(clf, 
#                    feature_names=X_original.columns.tolist(),  
#                    class_names=['0','1'],
#                    filled=True, fontsize=10)

# print(tree)

In [ ]:
df2 = df.copy()

In [ ]:
feature_names = df2.iloc[:,1:]
target_names = df.iloc[:,0]

In [ ]:
# plt.figure(figsize = (5,5), dpi = 800)

# tree.plot_tree(clf,feature_names = list(feature_names), \
#               class_names = ["0", "1"], proportion = True, \
#                filled = True)

In [ ]:
# !pip install graphviz

# from sklearn.tree import DecisionTreeClassifier
# from sklearn import datasets
# from IPython.display import Image  
# from sklearn import tree
# import pydotplus

# dot_data = tree.export_graphviz(clf, out_file=None, 
#                                 feature_names=list(feature_names.columns),  
# #                                 class_names=iris.target_names
#                                )

# # Draw graph
# graph = pydotplus.graph_from_dot_data(dot_data)  

# # Show graph
# print(graph)
# Image(graph.create_png())

In [ ]:
# Display the rules of all the nodes
plt.figure(figsize = None, dpi = 800)
tree_rules = tree.export_text(clf,  feature_names = list(feature_names))
print(tree_rules)

In [ ]:
# plt.figure(figsize = (5,5), dpi = 800)

# feature_names = df.drop("left", axis =1).columns

# tree.plot_tree(clf,feature_names = feature_names.tolist(), \
#               class_names = ["stayed", "left"], proportion = True, \
#                filled = True)

# plt.show

In [ ]:
# text_representation = tree.export_text(clf)
# print(text_representation)

In [ ]:
# from sklearn.tree import _tree

# def tree_to_code(tree, feature_names):

# 	'''
# 	Outputs a decision tree model as a Python function
	
# 	Parameters:
# 	-----------
# 	tree: decision tree model
# 		The decision tree to represent as a function
# 	feature_names: list
# 		The feature names of the dataset used for building the decision tree
# 	'''

# 	tree_ = tree.tree_
# 	feature_name = [
# 		feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
# 		for i in tree_.feature
# 	]
# 	print("def tree({}):".format(", ".join(feature_names)))

# 	def recurse(node, depth):
# 		indent = "  " * depth
# 		if tree_.feature[node] != _tree.TREE_UNDEFINED:
# 			name = feature_name[node]
# 			threshold = tree_.threshold[node]
# 			print("{}if {} <= {}:".format(indent, name, threshold))
# 			recurse(tree_.children_left[node], depth + 1)
# 			print("{}else:  # if {} > {}".format(indent, name, threshold))
# 			recurse(tree_.children_right[node], depth + 1)
# 		else:
# 			print("{}return {}".format(indent, tree_.value[node]))

# 	recurse(0, 1)

In [ ]:
# from sklearn.tree import _tree

# tree_to_code(df2, list(X_original))

### ------------------------------------------------------------End of Capstone---------------------------------------------------------------------
-----------